In [5]:
import sys
sys.path.append("..")


In [6]:
import datasets
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
)
from sentence_transformers.losses import MultipleNegativesRankingLoss
from sentence_transformers.training_args import BatchSamplers

In [7]:
# 1. Load a model to finetune
model = SentenceTransformer(
    "BAAI/bge-small-zh-v1.5"
)

In [13]:
from eval.dataset import RAGDataset
dataset = RAGDataset.from_file("../data/infgrad_retrieval_data_llm.json")
train_dataset, dataset_keys = dataset.get_train_dataset(split="train", negative_num=1, query_instruction_for_retrieval="为这个句子生成表示以用于检索相关文章：")
eval_dataset, _ = dataset.get_train_dataset(split="val", negative_num=1, query_instruction_for_retrieval="为这个句子生成表示以用于检索相关文章：")

In [14]:
for i in train_dataset[:2]:
    print(i)


{'query': '为这个句子生成表示以用于检索相关文章：美国旅游签证政策', 'pos': '美国旅游签证政策介绍：\n美国旅游签证（B-2 签证）是用于旅游、休闲和医疗等目的的签证。申请者需要向美国大使馆或领事馆提交 DS-160 表格，并支付相应的签证申请费用。\n\n对于旅游签证，申请者需要提供个人旅行计划、财务证明、护照以及其他支持文件。此外，申请者可能需要参加面试。\n\n美国旅游签证的有效期通常为六个月，但具体有效期将根据申请者的国籍和护照类型而定。持有 B-2 签证的游客可以在美国停留最多六个月，但这需要美国入境口岸的官员批准。\n\n值得注意的是，虽然 B-2 签证允许旅游和休闲活动，但禁止在美从事任何有偿工作。如果游客想要在美国工作，他们需要申请其他类型的签证。\n\n此外，受 COVID-19 疫情影响，美国旅游签证政策可能会有所变动。因此，申请者应该在计划前往美国之前，查阅最新的签证政策和入境要求。\n\n总的来说，美国旅游签证政策旨在允许外籍游客进入美国进行旅游和休闲活动，同时也确保了国家安全和经济利益。申请者需要准备好所有必要的文件，并了解最新的政策变化，以确保他们的旅行计划顺利进行。', 'neg_0': '英国旅游签证政策介绍：\n英国旅游签证（Standard Visitor visa）是用于旅游、休闲、参加商务会议、短期学习和医疗等目的的签证。申请者需要填写在线申请表并支付签证申请费用。\n\n英国旅游签证通常有效期为六个月，但最长可达两年。持有该签证的游客可以在英国停留最长六个月。\n\n申请者需要提供行程计划、住宿证明、财务证明以及其他支持文件。此外，申请者可能需要参加面试。\n\n值得注意的是，尽管英国旅游签证允许游客进行多种活动，但禁止在英从事任何有偿工作。如果游客想要在英国工作，他们需要申请其他类型的签证。\n\n由于英国脱欧，英国旅游签证政策可能会有所变动。因此，申请者应该在计划前往英国之前，查阅最新的签证政策和入境要求。\n\n总的来说，英国旅游签证政策旨在允许外籍游客进入英国进行旅游和休闲活动，同时也确保了国家安全和经济利益。申请者需要准备好所有必要的文件，并了解最新的政策变化，以确保他们的旅行计划顺利进行。'}
{'query': '为这个句子生成表示以用于检索相关文章：AI产品市场前景分析', 'pos': 

In [15]:
loss = MultipleNegativesRankingLoss(model)

In [23]:
args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir="../checkpoint/bge-small-zh-v1.5-sft",
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=16, # global batch size = 32 * 16 = 512
    learning_rate=2e-5,
    warmup_ratio=0.1,
    fp16=False,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=True,  # Set to True if you have a GPU that supports BF16
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # MultipleNegativesRankingLoss benefits from no duplicate samples in a batch
    # Optional tracking/debugging parameters:
    eval_strategy="steps",
    eval_steps=20,
    save_strategy="steps",
    save_steps=100000,
    save_total_limit=1,
    logging_steps=20,
    seed=42,
    lr_scheduler_type="cosine",
    optim="adamw_torch_fused",
)

In [24]:
import torch
from typing import Any

# fix transformer not compatible with sentence_transformers
class CustomSentenceTransformerTrainer(SentenceTransformerTrainer):
    def compute_loss(
        self,
        model: SentenceTransformer,
        inputs: dict[str, torch.Tensor | Any],
        return_outputs: bool = False,
        num_items_in_batch: int = None,
    ) -> torch.Tensor | tuple[torch.Tensor, dict[str, Any]]:
        return super().compute_loss(model, inputs, return_outputs)
       
    
trainer = CustomSentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=datasets.Dataset.from_list(train_dataset).select_columns(
        dataset_keys
    ),
    eval_dataset=datasets.Dataset.from_list(eval_dataset).select_columns(
        dataset_keys
    ),
    loss=loss,
)
trainer.train()

/home/cynic/miniconda3/envs/embedding/lib/python3.10/site-packages/sentence_transformers/trainer.py:205: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomSentenceTransformerTrainer.__init__`. Use `processing_class` instead.
  super().__init__(


Step,Training Loss,Validation Loss
20,0.649200,0.057490
40,0.571300,0.057718
60,0.675100,0.056017
80,0.658000,0.057280
100,0.664500,0.055631
120,0.585500,0.056261
140,0.744400,0.055054
160,0.773000,0.053215
180,0.889300,0.050928
200,0.974400,0.051949


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


TrainOutput(global_step=357, training_loss=0.7954315084035323, metrics={'train_runtime': 289.7227, 'train_samples_per_second': 631.566, 'train_steps_per_second': 1.232, 'total_flos': 0.0, 'train_loss': 0.7954315084035323, 'epoch': 0.9987760097919217})

In [27]:
model.save("../checkpoint/bge-small-zh-v1.5-sft")